In [2]:
!pip install pandas polars bs4

import pandas as pd
import polars as pl
import bs4
import requests
import re
import os

Defaulting to user installation because normal site-packages is not writeable


In [4]:
# get all links that contains .csv 
relative_path = './pmqc/'
url = 'https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/pmqc-programa-de-monitoramento-da-qualidade-dos-combustiveis'
response = requests.get(url)
soup = bs4.BeautifulSoup(response.text, 'html.parser')
links = soup.find_all('a', href=re.compile(r'.csv'))
# get only links between href and " target
links = [link['href'] for link in links if 'target' in link.attrs]

# # download all files using multiprocessing and urllib
import urllib.request
from multiprocessing.pool import ThreadPool

def download_file(link):
    # from string https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/pmqc/2023/pmqc_2023_04.csv
    # get year of file 
    year = link.split('/')[-2]
    name_file = link.split('/')[-1]
    if not os.path.exists(f"./pmqc/"):
        os.makedirs(f"./pmqc/")
    urllib.request.urlretrieve(link, f"./pmqc/{name_file}")

pool = ThreadPool(8)
pool.map(download_file, links)
pool.close()



ContentTooShortError: <urlopen error retrieval incomplete: got only 6422528 out of 16427637 bytes>

In [3]:
%%time 

if not os.path.exists(f"pmqc_processed/"):
    os.makedirs(f"pmqc_processed/")
    
df = pl.read_csv("pmqc/*.csv", separator=';', infer_schema_length=10000)#.limit(1000)
df = df.with_columns(
    pl.col('DataColeta').str.strptime(pl.Date, "%Y-%m-%d", strict=False).cast(pl.Date),
    # remove all special characters from column CnpjMatriz
    pl.col('CnpjPosto').str.replace_all(r'[^0-9]', '')
)
## print count rows
print(df.shape)

# # ## generate postos dimensions
postos = df.select([
    'CnpjPosto',
    'RazaoSocialPosto',
    'Distribuidora',
    'Endereço',
    'Latitude',
    'Longitude',
    'Bairro',
    'Município',
    'Uf'])
postos = postos.unique(subset=["CnpjPosto"])
postos = postos.with_columns(
    pl.col('CnpjPosto').str.slice(0, 8).cast(pl.Int32).alias('CnpjMatriz')
)
# filter postos in MS
postos = postos.filter(pl.col('Uf') == 'MS')
# limit 300 postos
postos = postos.limit(2000)
postos.write_json('pmqc_processed/postos.json', row_oriented=True)



(5761345, 19)
CPU times: user 12.5 s, sys: 8.39 s, total: 20.8 s
Wall time: 6.97 s


In [4]:
%%time

# generate fact table
coletas = df.select([
    'DataColeta',
    'IdNumeric',
    'CnpjPosto',
    'Produto',
    'Ensaio',
    'Resultado',
    'UnidadeEnsaio',
    'Conforme'])
coletas = coletas.unique(subset=['IdNumeric'])
# get only postos in postos dataframe
coletas = coletas.join(postos, on='CnpjPosto', how='inner')
coletas.write_json('pmqc_processed/coletas.json', row_oriented=True)


CPU times: user 1.65 s, sys: 128 ms, total: 1.78 s
Wall time: 330 ms
